In [2]:
from pyalex import (
    Works, Authors, Sources,
    Institutions, Concepts, Publishers, Funders
)
import pyalex
import pandas as pd
import numpy as np
pyalex.config.email = "david@rs21.io"

from flair.embeddings import DocumentPoolEmbeddings
from flair.data import Sentence
from flair.embeddings import SentenceTransformerDocumentEmbeddings

EMBEDDING_MODEL_1 = "all-mpnet-base-v2" 

# this one is also good: all-MiniLM-L6-v2
EMBEDDING_MODEL_2 = "all-MiniLM-L6-v2"
SENT_EMBEDDINGS_1 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_1)
SENT_EMBEDDINGS_2 = SentenceTransformerDocumentEmbeddings(EMBEDDING_MODEL_2)
DOC_EMBEDDINGS= DocumentPoolEmbeddings([SENT_EMBEDDINGS_2])

import torch
from tqdm import tqdm
import yake
import umap.umap_ as umap
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import altair as alt
import math
import plotly.express as px
import textwrap
import streamlit as st

In [3]:
import os
os.environ["OPENAI_API_KEY"] = st.secrets["OPENAI_API_KEY"]

In [4]:
search_term = 'jamming'
search_term = 'radar jamming and deception|electronic warfare|Network-centric warfare|Air-to-air missile'
search_term = ('radar jamming and deception|electronic warfare|Network-centric warfare|missile guidance' +
               '|Robot manipulator|Automatic target recognition' + 
              '|Pulsed power|High-energy X-rays' + 
              '|Ballistic missile|Air-to-air missile' + 
              '|terminal guidance')
jamming_concepts = Concepts().\
search_filter(display_name=search_term).get()

In [5]:
concepts = []
for i in range(len(jamming_concepts)):
    id_, display_name = jamming_concepts[i]['id'], jamming_concepts[i]['display_name']
    concepts.append((id_, display_name))
concepts

[('https://openalex.org/C2985527887', 'Robot manipulator'),
 ('https://openalex.org/C97039730', 'Pulsed power'),
 ('https://openalex.org/C522053795', 'Missile guidance'),
 ('https://openalex.org/C117623542', 'Automatic target recognition'),
 ('https://openalex.org/C122136912', 'Ballistic missile'),
 ('https://openalex.org/C183838350', 'High-energy X-rays'),
 ('https://openalex.org/C176381164', 'Radar jamming and deception'),
 ('https://openalex.org/C133082901', 'Electronic warfare'),
 ('https://openalex.org/C2777047555', 'Terminal guidance'),
 ('https://openalex.org/C2781187084', 'Network-centric warfare'),
 ('https://openalex.org/C202802212', 'Air-to-air missile')]

In [6]:
def process_works_list(worklist:list):
    """
    transforms the 
    works list into a dataframe.
    """
    abstracts_dict = {h["id"]:h["abstract"] for h in worklist}
    df = pd.DataFrame.from_records(worklist)
    del df['abstract_inverted_index']
    df['abstract'] = df['id'].map(abstracts_dict)
   # df['author_affils'] = df['authorships'].apply(get_authors_and_affils)
    return df

In [7]:
for i in range(len(jamming_concepts)):
    print(jamming_concepts[i]['id'], jamming_concepts[i]['works_count'])


https://openalex.org/C2985527887 9898
https://openalex.org/C97039730 13302
https://openalex.org/C522053795 6247
https://openalex.org/C117623542 3360
https://openalex.org/C122136912 6049
https://openalex.org/C183838350 1134
https://openalex.org/C176381164 2593
https://openalex.org/C133082901 3254
https://openalex.org/C2777047555 1364
https://openalex.org/C2781187084 1808
https://openalex.org/C202802212 1322


In [8]:
def get_concept_frame(concepts_list:list, i:int):
    """
    takes a list of Concepts() results and an index
    and forms the pagination object to retrive the 
    records
    """
    pager = Works().filter(publication_year='>2016',
    #concepts={"id":f"{concepts_list[i]['id']}"}).filter(authorships={"institutions":{"country_code":"CN"}}).\
    #paginate(per_page=200,n_max=None)
    concepts={"id":f"{concepts_list[i]['id']}"}).\
    paginate(per_page=200,n_max=None)
    df = pd.DataFrame()
    for page in tqdm(pager):
        dfpage = process_works_list(page)
        df = pd.concat([df, dfpage], ignore_index=True)
        df.drop_duplicates(subset='id', keep='first',inplace=True)
    return df

In [9]:
frames_list = []
for i in range(len(jamming_concepts)):
    df = get_concept_frame(jamming_concepts, i)
    frames_list.append(df)

12it [00:22,  1.86s/it]
13it [00:21,  1.68s/it]
6it [00:10,  1.72s/it]
6it [00:10,  1.76s/it]
5it [00:08,  1.62s/it]
1it [00:01,  1.84s/it]
4it [00:06,  1.64s/it]
4it [00:06,  1.62s/it]
2it [00:03,  1.75s/it]
2it [00:03,  1.66s/it]
1it [00:01,  1.54s/it]


In [10]:
dftop = pd.concat(frames_list,
                  ignore_index=True)
dftop.drop_duplicates(subset='id', keep='first', 
                      inplace=True)

dftop.set_index('id', inplace=True, drop=False)

dfall = dftop
print(dfall.shape)

dfall['content'] = dfall['title'] + ". " + dfall['abstract']

dfrecords = dfall[~dfall['content'].isna()].copy()

(9748, 42)


In [11]:
def get_keywords(text:str, top:int=7, stopwords=None):
    """
    takes a blob of text and 
    returns the top **top** 
    keywords as a list
    """
    kw_extractor = yake.KeywordExtractor(top=top, stopwords=stopwords)
    keywords = kw_extractor.extract_keywords(text)
    return [p[0] for p in keywords]

In [12]:
def get_top_concepts(concept_list:list,score:float=.6):
    """
    takes a list of concept dictionaries 
    returns the top **top** display_names;
    concepts whose score is >= score
    """
    return [c['display_name'] for c in concept_list if c['score'] >= score]

In [13]:
dfrecords['keywords'] = dfrecords['content'].apply(get_keywords)
dfrecords['top_concepts'] = dfrecords['concepts'].apply(get_top_concepts)

In [14]:
texts = dfrecords['content'].str.lower().values.tolist()
#dfrecords.to_csv('jamming.csv')

In [15]:
def get_content_embeddings(dfrecords:pd.DataFrame) -> pd.DataFrame:
    """
    passes the preprocessed mitigation strings
    data through the embedding model to produce the vector
    space representation of each pet mitigation.
    """
    sent = Sentence("The grass is green.")
    DOC_EMBEDDINGS.embed(sent)
    texts = dfrecords["content"].str.lower().values.tolist()
    all_descriptions = np.empty((len(texts), len(sent.embedding)))
    for i in tqdm(range(len(texts))):
        sent = Sentence(texts[i])
        DOC_EMBEDDINGS.embed(sent)
        all_descriptions[i, :] = sent.embedding.cpu().numpy()
        # gc.collect()
        torch.cuda.empty_cache()
    dfcontentvectors = pd.DataFrame.from_records(all_descriptions, index=dfrecords.index)
    return dfcontentvectors

In [16]:
dfcontentvectors = get_content_embeddings(dfrecords)

100%|█████████████████████████████████████████████████████████████████████| 8045/8045 [01:50<00:00, 72.91it/s]


In [17]:
#umap.UMAP?
N_COMPONENTS = 2 # can visualize this way
umap_reducer = umap.UMAP(n_components=N_COMPONENTS,
                       #  metric='euclidean')
                         random_state=1234,
                         metric='cosine')  # can experiment with this metric as well as the other 
# parameters
# to see what other literature is in the same information space, we need to keep this umap_reducer 
# object as well as the gmm model below.

# Apply UMAP to the vectorized strings
reduced_vectors = umap_reducer.fit_transform(dfcontentvectors.to_numpy())
dfreduced = pd.DataFrame.from_records(reduced_vectors, 
                index=dfcontentvectors.index)
dfreduced.columns = ['x','y']

# use hdbscan to cluster

In [18]:
import hdbscan

hdbscan_args = {'min_cluster_size': 15,
                            'metric': 'euclidean',
                            'cluster_selection_method': 'eom',
                            'cluster_selection_epsilon': 0.1
               }

cluster = hdbscan.HDBSCAN(**hdbscan_args).fit(dfreduced[['x','y']].to_numpy())

dfreduced['cluster'] = cluster.labels_
dfreduced['probability'] = cluster.probabilities_

dfpapers = dfrecords.merge(dfreduced, left_index=True,
                           right_index=True)

In [19]:
#help(dfpapers.explode)
del dfpapers['id']
dfstart = dfpapers.reset_index()
dfstart.head()

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,...,fulltext_origin,abstract,is_authors_truncated,content,keywords,top_concepts,x,y,cluster,probability
0,https://openalex.org/W1517236425,https://doi.org/10.1201/9781003062714,Neural Network Control Of Robot Manipulators A...,Neural Network Control Of Robot Manipulators A...,2020,2020-08-13,{'openalex': 'https://openalex.org/W1517236425...,en,"{'is_oa': False, 'landing_page_url': 'https://...",book,...,NaN,"There has been great interest in ""universal co...",NaN,Neural Network Control Of Robot Manipulators A...,"[Robot Manipulators, Neural network controller...","[Robot manipulator, Artificial neural network]",12.853262,-5.100251,3,1.0
1,https://openalex.org/W2787504052,https://doi.org/10.1109/tie.2018.2803773,Adaptive Parameter Estimation and Control Desi...,Adaptive Parameter Estimation and Control Desi...,2018,2018-10-01,{'openalex': 'https://openalex.org/W2787504052...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,ngrams,For parameter identifications of robot systems...,NaN,Adaptive Parameter Estimation and Control Desi...,"[Adaptive Parameter Estimation, Control Design...","[Convergence (economics), Control theory (soci...",12.647867,-5.867312,3,1.0
2,https://openalex.org/W2591194352,https://doi.org/10.1109/tie.2017.2674624,Manipulability Optimization of Redundant Manip...,Manipulability Optimization of Redundant Manip...,2017,2017-06-01,{'openalex': 'https://openalex.org/W2591194352...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,ngrams,For solving the singularity problem arising in...,NaN,Manipulability Optimization of Redundant Manip...,"[dynamic neural network, Manipulability Optimi...",[],12.589683,-4.873707,3,1.0
3,https://openalex.org/W2418767125,https://doi.org/10.1109/tsmc.2016.2562506,Neural Network Control of a Flexible Robotic M...,Neural Network Control of a Flexible Robotic M...,2017,2017-08-01,{'openalex': 'https://openalex.org/W2418767125...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,ngrams,Adaptive neural networks (NNs) are employed fo...,NaN,Neural Network Control of a Flexible Robotic M...,"[Flexible Robotic Manipulator, Flexible Roboti...",[Spring (device)],12.971470,-5.318573,3,1.0
4,https://openalex.org/W2740675802,https://doi.org/10.1109/tcyb.2017.2711961,Adaptive Neural Network Control of a Robotic M...,Adaptive Neural Network Control of a Robotic M...,2017,2017-10-01,{'openalex': 'https://openalex.org/W2740675802...,en,"{'is_oa': False, 'landing_page_url': 'https://...",article,...,ngrams,The control problem of an uncertain n -degrees...,NaN,Adaptive Neural Network Control of a Robotic M...,"[Time-Varying Output Constraints, Output Const...","[Control theory (sociology), Lyapunov function...",13.228225,-5.346364,3,1.0


In [20]:
dfbig = dfstart.explode(column='authorships')
dfbig.shape, dfstart.shape

((32209, 49), (8045, 49))

In [21]:
def add_extra_to_authorships(row: pd.DataFrame):
    """
    row[authorships] is a dictionary;
    add in the id key to that dictionary
    whose value is row[id]
    """
    complete_dict = row["authorships"]
   # assert type(complete_dict) == dict
    #print(type(complete_dict))
    if type(complete_dict) == dict:
        complete_dict["id"] = row["id"]
        complete_dict["x"] = row["x"]
        complete_dict["y"] = row["y"]
        complete_dict["cluster"] = row["cluster"]
        complete_dict["cluster_score"] = row["probability"]
        complete_dict["title"] = row["title"]
        complete_dict["abstract"] = row["abstract"]
        complete_dict["doi"] = row["doi"]
        complete_dict["publication_date"] = row["publication_date"]
        complete_dict["publication_year"] = row["publication_year"]
        complete_dict["grants"] = row["grants"]
        complete_dict["locations"] = row["locations"]
        return complete_dict
    else:
        return row["authorships"]

In [22]:
dfbig['big_authorships'] = dfbig.apply(add_extra_to_authorships, axis=1)

In [23]:
#dfbig['authorships'].tolist()
bigvals = dfbig['authorships'].tolist()

In [24]:
dictvals = [c for c in bigvals if type(c) != float]


In [25]:
dftriple = pd.json_normalize(dictvals,
                  record_path=['institutions'],
                  meta=['id','raw_affiliation_string','author_position', 'doi',
                        'title','abstract','publication_date', 'publication_year',
                        'grants','locations',
                        'is_corrresponding','x','y','cluster','cluster_score',
                       ['author','id'], ['author', 'display_name'],
                       ['author','orcid']],
                  errors='ignore',
                  sep='_',
                  meta_prefix='paper_',
                #  record_prefix='author_'
                 )

In [26]:
dfpapers.columns

Index(['doi', 'title', 'display_name', 'publication_year', 'publication_date',
       'ids', 'language', 'primary_location', 'type', 'type_crossref',
       'open_access', 'authorships', 'countries_distinct_count',
       'institutions_distinct_count', 'corresponding_author_ids',
       'corresponding_institution_ids', 'apc_list', 'apc_paid', 'has_fulltext',
       'cited_by_count', 'biblio', 'is_retracted', 'is_paratext', 'concepts',
       'mesh', 'locations_count', 'locations', 'best_oa_location',
       'sustainable_development_goals', 'grants', 'referenced_works_count',
       'referenced_works', 'related_works', 'ngrams_url', 'cited_by_api_url',
       'counts_by_year', 'updated_date', 'created_date', 'fulltext_origin',
       'abstract', 'is_authors_truncated', 'content', 'keywords',
       'top_concepts', 'x', 'y', 'cluster', 'probability'],
      dtype='object')

In [27]:
dftopics = dfcontentvectors.copy()
dftopics['cluster'] = dfpapers['cluster']
dfmeantopics = dftopics.groupby('cluster').mean().copy()
reduced_topics = umap_reducer.transform(dfmeantopics.to_numpy())
df_reduced_topics = pd.DataFrame.from_records(reduced_topics, 
                index=dfmeantopics.index)
df_reduced_topics.columns = ['x','y']
df_reduced_topics['topic'] = df_reduced_topics.index
df_reduced_topics.head()

def get_cluster_concepts(topic_num:int, n:int=20):
    """
    takes an integer topic_num corresponding to a 
    given topic number and
    returns the list of top n occuring concepts
    from the top_concept field
    """
    top_concepts = dfpapers[dfpapers['cluster'] == topic_num]['top_concepts'].tolist()
    flat_concepts = [item for sublist in top_concepts for item in sublist]
    concepts_dict = {c:flat_concepts.count(c) for c in flat_concepts}
    sorted_concepts = sorted(concepts_dict.items(), key=lambda x:x[1], reverse=True)
    return [c[0] for c in sorted_concepts][:n]

def get_yake_cluster_phrases(topic_num:int, n:int=20):
    """
    takes in an integer n corresponding
    to a given topic number and
    returns the list of keyphrases (TopicRank method)
    """
    documents = dfpapers[dfpapers['cluster'] == topic_num]['content'].tolist()
    topic_input = ". ".join(documents)
    #extractor = pke.unsupervised.TextRank()
    kw_extractor = yake.KeywordExtractor(top=n, stopwords=None)
    keywords = kw_extractor.extract_keywords(topic_input)
    #extractor.load_document(input=topic_input,
    #                    language='en',
    #                    normalization=None)

    #extractor.candidate_selection()

    #window = 2
    #use_stems = False
    #extractor.candidate_weighting(window=window,
    #                          use_stems=use_stems)
    #extractor.candidate_weighting()
    #threshold = 0.8
   # keyphrases = extractor.get_n_best(n=20, threshold=threshold)
    #keyphrases = extractor.get_n_best(n=n)
    return [p[0] for p in keywords]

wikiconcepts = df_reduced_topics['topic'].apply(get_cluster_concepts)

wikikeywords = df_reduced_topics['topic'].apply(get_yake_cluster_phrases)

dfpapers['id'] = dfpapers.index
dfinfo = dfpapers[['x','y','id','title','abstract','doi','cluster','grants',
                   'locations',
                 'publication_date','keywords','top_concepts']].copy()

centroids = dfinfo.groupby('cluster')[['x','y']].mean().copy()
centroids['concepts'] = wikiconcepts
centroids['cluster'] = centroids.index
centroids['keywords'] = wikikeywords

In [28]:
def wrap_it(x):
    return "<br>".join(textwrap.wrap(x, width=40))
   # return "<br>".join(textwrap.wrap(x.replace(r'\s+', ' '), width=40))

In [29]:
centroids['wrapped_keywords'] = centroids['keywords'].apply(str).apply(wrap_it)
centroids['wrapped_concepts'] = centroids['concepts'].apply(str).apply(wrap_it)

In [30]:
centroids.to_pickle('jammingcentroids2d.pkl')

In [31]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_string', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [32]:
dftriple.to_pickle('jammingdftriple2d.pkl')


In [33]:
def get_affils_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by id, ror sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg.groupby(['id','display_name','country_code',
                     'type'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [34]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 1)
print(kw84)
dv84.head(10)

1
['Aerial Manipulator', 'Aerial Robotic Manipulators', 'Aerial', 'Aerial Manipulator System', 'Aerial Manipulator Robot', 'Manipulator', 'unmanned aerial vehicle', 'Aerial Robotic', 'aerial manipulator control', 'Control Aerial Manipulator', 'aerial manipulator vehicle', 'Protocentric Aerial Manipulators', 'robotic manipulator', 'proposed aerial manipulator', 'control', 'unmanned aerial', 'Aerial Continuum Manipulator', 'cable-Suspended Aerial Manipulator', 'aerial vehicle', 'Overactuated Aerial Manipulators']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I2898391981,German Aerospace Center,DE,facility,6.573864
https://openalex.org/I157773358,Sun Yat-sen University,CN,education,6.0
https://openalex.org/I118946981,Escuela Politécnica del Ejército,EC,education,5.851659
https://openalex.org/I17866349,Université de Toulouse,FR,education,5.802707
https://openalex.org/I1294671590,French National Centre for Scientific Research,FR,government,5.802707
https://openalex.org/I201537933,Tohoku University,JP,education,5.525794
https://openalex.org/I79238269,Universidad de Sevilla,ES,education,4.39277
https://openalex.org/I4210098188,Laboratori Guglielmo Marconi (Italy),IT,company,4.303968
https://openalex.org/I2801039369,Marconi University,IT,education,4.303968


In [35]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 0)
print(kw84)
dv84.head(10)

0
['energy storage density', 'energy storage', 'energy storage performance', 'high energy storage', 'high power density', 'energy storage properties', 'recoverable energy storage', 'energy density', 'excellent energy storage', 'discharge energy density', 'energy storage ceramics', 'energy', 'recoverable energy density', 'power density', 'storage density', 'High energy density', 'energy storage efficiency', 'dielectric energy storage', 'high energy', 'storage']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I87445476,Xi'an Jiaotong University,CN,education,45.412704
https://openalex.org/I109935558,Ningbo University,CN,education,28.963583
https://openalex.org/I139024713,Guangdong University of Technology,CN,education,26.716671
https://openalex.org/I150229711,University of Electronic Science and Technology of China,CN,education,26.260822
https://openalex.org/I4210124847,National Engineering Research Center of Electromagnetic Radiation Control Materials,CN,facility,25.260822
https://openalex.org/I143868143,Anhui University,CN,education,22.024599
https://openalex.org/I116953780,Tongji University,CN,education,20.58211
https://openalex.org/I1297991670,Southwest University of Science and Technology,CN,education,19.497119
https://openalex.org/I142108993,Southwest University,CN,education,17.336764


In [36]:
dv84, kw84 = get_affils_cluster_sort(dftriple, 2)
print(kw84)
dv84.head(20)

2
['robotic apple harvesting', 'apple harvesting robot', 'robotic harvesting system', 'apple harvesting', 'robotic', 'robotic manipulator', 'robotic system', 'Harvesting Manipulators', 'harvesting', 'manipulator', 'harvesting robot', 'robotic apple', 'Picking', 'robot', 'robotic harvesting', 'apple harvesting prototype', 'system', 'Based Robotic Manipulator', 'control', 'Harvesting Robotic System']


,,,,paper_cluster_score
id,display_name,country_code,type,
https://openalex.org/I4210166615,Institute for Systems and Computer Engineering of Porto,PT,nonprofit,15.808033
https://openalex.org/I87216513,Michigan State University,US,education,8.0
https://openalex.org/I183898223,International Hellenic University,GR,education,8.0
https://openalex.org/I155381812,University of Trás-os-Montes and Alto Douro,PT,education,5.0
https://openalex.org/I115592961,Jiangsu University,CN,education,4.139129
https://openalex.org/I3130994585,Antenor Orrego Private University,PE,education,4.0
https://openalex.org/I182534213,University of Porto,PT,education,3.523213
https://openalex.org/I4210159532,Federal Scientific Agroengineering Center VIM,RU,facility,3.0
https://openalex.org/I4210123755,Vietnam National University of Forestry,VN,education,3.0


In [37]:
dfinfo = dfpapers[['x','y','id','title','abstract', 'doi','cluster','probability',
                 'publication_date','grants','locations',
                   'keywords','top_concepts']].copy()

In [38]:
pap_affils_dict = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: ' | '.join(x.tolist()))

#pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].\
#apply(lambda x: ' | '.join(x.tolist()))

In [39]:
pap_authors_dict = dftriple.groupby('paper_id')['paper_author_display_name'].apply(lambda x: x.values)

In [40]:
dfinfo['affil_list'] = pap_affils_dict
dfinfo['author_list'] = pap_authors_dict

In [41]:
dfinfo['affil_list'] = dftriple.groupby('paper_id')['paper_raw_affiliation_string'].\
apply(lambda x: x.tolist())

In [42]:
dfinfo['author_list'] =  dftriple.groupby('paper_id')['paper_author_display_name'].\
apply(lambda x: x.tolist())

In [43]:
dfinfo['wrapped_affil_list'] = dfinfo['affil_list'].apply(str).apply(wrap_it)
dfinfo['wrapped_author_list'] = dfinfo['author_list'].apply(str).apply(wrap_it)

In [44]:
dfinfo['wrapped_keywords'] = dfinfo['keywords'].apply(str).apply(wrap_it)

In [45]:
def get_source_name(loc_list):
    """
    grab the first item in the list;
    retturn the display name
    """
    try:
        primary = loc_list[0]
        return primary["source"]["display_name"]
    except:
        return None

def get_source_type(loc_list):
    """
    grab the first item in the list;
    return the source type
    """
    try:
        primary = loc_list[0]
        return primary["source"]["type"]
    except:
        return None
    

In [46]:
dfinfo["source"] = dfinfo["locations"].apply(get_source_name)
dfinfo["source_type"] = dfinfo["locations"].apply(get_source_type)

In [47]:
dfinfo.columns

Index(['x', 'y', 'id', 'title', 'abstract', 'doi', 'cluster', 'probability',
       'publication_date', 'grants', 'locations', 'keywords', 'top_concepts',
       'affil_list', 'author_list', 'wrapped_affil_list',
       'wrapped_author_list', 'wrapped_keywords', 'source', 'source_type'],
      dtype='object')

In [48]:
dftriple.columns

Index(['id', 'display_name', 'ror', 'country_code', 'type', 'lineage',
       'paper_id', 'paper_raw_affiliation_string', 'paper_author_position',
       'paper_doi', 'paper_title', 'paper_abstract', 'paper_publication_date',
       'paper_publication_year', 'paper_grants', 'paper_locations',
       'paper_is_corrresponding', 'paper_x', 'paper_y', 'paper_cluster',
       'paper_cluster_score', 'paper_author_id', 'paper_author_display_name',
       'paper_author_orcid'],
      dtype='object')

In [49]:
dfinfo.to_pickle('jammingdfinfo2d.pkl')

In [50]:
def get_funder_names(funder_list):
    """
    funder_list is a list of dictionaries
    with three keys; return the list of 
    unique **funder_display_name**
    values
    """
    try:
        funder_names = list(set([f['funder_display_name'] for f in funder_list]))
        return funder_names    
    except:
        return []

In [51]:
dftriple["source"] = dftriple["paper_locations"].apply(get_source_name)
dftriple["source_type"] = dftriple["paper_locations"].apply(get_source_type)
dftriple["funder_list"] = dftriple["paper_grants"].apply(get_funder_names)

In [52]:
dftriple.to_pickle('jammingdftriple2d.pkl')

In [53]:
dfinfo["funder_list"] = dfinfo["grants"].apply(get_funder_names)
dfinfo["wrapped_funder_list"] = dfinfo["funder_list"].apply(str).apply(wrap_it)

In [54]:
dfinfo.to_pickle('jammingdfinfo2d.pkl')

In [55]:
kw_dict = dfinfo['keywords'].to_dict()

In [56]:
sources_list = dftriple['source'].unique().tolist()
type(sources_list), len(sources_list)

(list, 1083)

In [57]:
def get_source_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    source_json = Sources().search_filter(display_name = s).get()
    a = source_json[0]['type']
    if "homepage_url" in source_json[0] and source_json[0]['homepage_url']:
        print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return source_json[0]["homepage_url"]
    else:
        return None

In [58]:
Sources.filter?

Signature: Sources.filter(self, **kwargs)
Docstring: <no docstring>
File:      ~/.local/lib/python3.10/site-packages/pyalex/api.py
Type:      function


In [59]:
sources_list[-56]

'Indian journal of information sources and services'

In [60]:
sj0 = get_source_json(sources_list[0])
sj0

IEEE Transactions on Industrial Electronics has homepage_url and type journal


'http://ieeexplore.ieee.org/servlet/opac?punumber=41'

In [61]:
#source_json

In [62]:
def get_display_page_dict(sl:list):
    """
    sl is a list of Sources display_name values
    returns the dictionary mapping
    display_names with homepage_url values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_source_json(s)
        except:
            pass
    return mapping_dict

In [63]:
source_page_dict = get_display_page_dict(sources_list)

  0%|                                                                        | 1/1083 [00:00<09:33,  1.89it/s]

IEEE Transactions on Industrial Electronics has homepage_url and type journal


  0%|▏                                                                       | 3/1083 [00:01<10:35,  1.70it/s]

IEEE transactions on cybernetics has homepage_url and type journal


  0%|▎                                                                       | 4/1083 [00:02<09:55,  1.81it/s]

...

Sistemi ozbroênnâ ì vìjsʹkova tehnìka has homepage_url and type journal


100%|████████████████████████████████████████████████████████████████████▉| 1082/1083 [08:32<00:00,  2.06it/s]

Aviation has homepage_url and type journal


100%|█████████████████████████████████████████████████████████████████████| 1083/1083 [08:32<00:00,  2.11it/s]

Frontiers in space technologies has homepage_url and type journal


In [64]:
import pickle

with open("source_page_dict.pkl", "wb") as f:
    pickle.dump(source_page_dict, f)

In [65]:
with open("source_page_dict.pkl", "rb") as f:
    source_dict = pickle.load(f)

In [66]:
def get_journals_cluster_sort(dc:pd.DataFrame, cl:int):
    """
    restricts the dataframe dc to cluster value cl
    and returns the results grouped by source (where
    source_type == 'journal') sorted
    by the some of probablity descending
    """
    dg = dc[dc['paper_cluster'] == cl].copy()
    print(cl)
    dv = dg[dg['source_type'] == 'journal'].groupby(['source'])['paper_cluster_score'].sum().to_frame()
    dv.sort_values('paper_cluster_score', ascending=False, inplace=True)
    dv['journal'] = dv.index
    dv['hompage_url'] = dv['journal'].map(source_dict)
    kw = centroids[centroids.cluster == cl]['keywords'].iloc[0]
    return dv, kw

In [68]:
dv, kw = get_journals_cluster_sort(dftriple, 2)
dv.head()

2


,paper_cluster_score,journal,hompage_url
source,,,
Industrial Robot-an International Journal,11.0,Industrial Robot-an International Journal,None
Mechatronics,7.0,Mechatronics,None
Computers and Electronics in Agriculture,5.139129,Computers and Electronics in Agriculture,https://www.journals.elsevier.com/computers-an...
Electronics,2.64805,Electronics,http://www.theiet.org/eletters
"Robotikusu, Mekatoronikusu Koenkai koen gaiyoshu",1.825191,"Robotikusu, Mekatoronikusu Koenkai koen gaiyoshu",NaN


In [69]:
affils_list = dftriple['display_name'].unique().tolist()
type(affils_list), len(affils_list)

affil_json = Institutions().filter(display_name = affils_list[1]).get()

def get_affil_json(s:str):
    """
    s is an openalex Sources display_name
    return that Sources object
    """
    affil_json = Institutions().search_filter(display_name = s).get()
    #a = source_json[0]['type']
    if "geo" in affil_json[0]:
       # print(f"{s} has homepage_url and type {source_json[0]['type']}")
        return affil_json[0]["geo"]["latitude"], affil_json[0]["geo"]["longitude"]
    else:
        return None, None
    
def get_display_geo_dict(sl:list):
    """
    sl is a list of Institution display_name values
    returns the dictionary mapping
    display_names with (latitude, longitude) values."""
    mapping_dict = dict()
    for s in tqdm(sl):
        try:
            mapping_dict[s] = get_affil_json(s)
        except:
            pass
    return mapping_dict

affil_geo_dict = get_display_geo_dict(affils_list)


import pickle

with open("affil_geo_dict.pkl", "wb") as f:
    pickle.dump(affil_geo_dict, f)

100%|█████████████████████████████████████████████████████████████████████| 2758/2758 [21:18<00:00,  2.16it/s]


In [98]:
#affil_geo_dict
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [99]:
llm = ChatOpenAI(model_name = 'gpt-4', # 'gpt-3.5-turbo', # 'text-davinci-003' , 'gpt-3.5-turbo'
             temperature=0.6,
             max_tokens=600)

In [101]:
topic_template = """
I want you to act as a naming consultant for scientific topics based on keyphrases.

Return a brief but detailed description of the scientific topic and applications related to
the scientific field desribed by the list of keyphrases and article titles. The description should be meaningful to an
new intelligence analyst. Highlight typical applications. Highlight any dual use technologies that may be of concern to the United States
Government.

What is a good summary of the scientific topic related to {topic_phrases} with article titles {topic_titles}?
Provide the summary in about 80 words. 
Please end with a complete sentence.
"""

prompt_topic = PromptTemplate(
    input_variables=["topic_phrases","topic_titles"],
    template=topic_template,
)

chain_topic= LLMChain(llm=llm, prompt=prompt_topic)



def get_topic_llm_description(key_phrases:list, key_titles:list):
    """
    takes in the key_phrases list
    and returns the openai returned description.
    """
    topic_phrases = ", ".join(key_phrases)
    topic_titles = ", ".join(key_titles)
    return chain_topic.run(topic_phrases=topic_phrases)

In [103]:
import openai
print(openai.__version__)

0.27.7
